In [6]:
import pandas as pd
from chatnoir_api.chat import ChatNoirChatClient

chatnoir_chat = ChatNoirChatClient()

ChatNoir Chat uses ws_host from environment Environment variable
ChatNoir Chat uses API key from Environment variable
ChatNoir Chat uses model 'alpaca-en-7b' from Environment variable
ChatNoir Chat uses endpoint 'https://chat.web.webis.de/' from {endpoint[1]}


In [4]:
documents = [
    {'docno': 'd1', 'text': 'The Golden Retriever is a Scottish breed of medium size.'},
    {'docno': 'd2', 'text': 'Intelligent types of dogs are: (1) Border Collies, (2) Poodles, and (3) German Shepherds.'},
    {'docno': 'd3', 'text': 'Poodles are a highly intelligent, energetic, and sociable.'},
    {'docno': 'd4', 'text': 'The European Shorthair is medium-sized to large cat with a well-muscled chest.'},
    {'docno': 'd5', 'text': 'The domestic canary is a small songbird.'}
]

topics = pd.DataFrame([
    {'qid': '1', 'query': 'dog'},
])

qrels = pd.DataFrame([
    {'qid': '1', 'docno': 'd1', 'relevance': 1}, #d1 is about an specific dog
    {'qid': '1', 'docno': 'd2', 'relevance': 1}, #d1 is about multiple types of dogs
    {'qid': '1', 'docno': 'd3', 'relevance': 1}, #d1 is about an specific dog
])

In [3]:
import pyterrier as pt
import pandas as pd
pd.set_option('display.max_colwidth', 0)

if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, blocks=True, meta={'docno': 100, 'text': 20480}, )
index_ref = indexer.index(documents)
index = pt.IndexFactory.of(index_ref)


In [7]:
chatnoir_chat.chat('How are you?')

"I'm doing well, thank you for asking. How about you?"

In [8]:
chatnoir_chat.chat('I have a search engine for animals. Please name specific breeds relevant for the query "dog".')

'Labrador Retriever, German Shepherd, Golden Retriever, Poodle, Rottweiler, Yorkshire Terrier, Boxer, Bulldogs, Chihuahua, Siberian Husky, Doberman Pinscher, Great Dane, Cocker Spaniel, St. Bernard, Pug, German Shorthaired Pointer, Dachshund, Maltese, Siberian Husky, Chihuahua, Labrador Retriever, German Shepherd, Golden Retriever, Boxer, Bulldogs, Yorkshire Terrier, Great Dane, Cocker Spaniel, St. Bernard, Pug, German Shorthaired Pointer, Dachshund, Maltese.'

In [9]:
def pseudo_document(query):
    # return only first 100 characters ~= 10 terms. Hypothesis: the LLM outputs more important terms first.
    # (this is cherry picked)
    return chatnoir_chat.chat(f'I have a search engine for animals. Please name specific breeds relevant for the query "{query}".')[:100]

In [10]:
# we only have one query for which we generate a pseudo relevant document
llm_expansion_documents = [
    {'docno': 'llm-expansion-for-query-1', 'text':  pseudo_document('dog')},
]


In [11]:
indexer = pt.IterDictIndexer('/tmp/llm-expansion-index', overwrite=True, blocks=True, meta={'docno': 100, 'text': 20480})
index_ref = indexer.index(llm_expansion_documents)
llm_index = pt.IndexFactory.of(index_ref)

In [17]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

bm25_bo1 = bm25 >> pt.rewrite.Bo1QueryExpansion(index) >> bm25

In [15]:
# We make a pyterrier-transformer out o fthe expansion documents 
# so that we can use it in subsequent pipelines.
llm_expansion = pt.Transformer.from_df(pd.DataFrame([
    {'qid': '1', 'docno': 'llm-expansion-for-query-1'},
]))

bm25_llm_bo1 = llm_expansion >> pt.rewrite.Bo1QueryExpansion(llm_index) >> bm25

In [18]:
pt.Experiment([bm25, bm25_bo1, bm25_llm_bo1], topics, qrels, eval_metrics=['ndcg_cut_5'], names=['BM25', 'BM25 >> Bo1 >> BM25', 'Alpaca >> Bo1 >> BM25'])

,name,ndcg_cut_5
0,BM25,0.469279
1,BM25 >> Bo1 >> BM25,0.765361
2,Alpaca >> Bo1 >> BM25,1.000000
